# Analyse des destinations navigo

Note préliminaire :

* dans cette analyse préliminaire on se fonde sur les *pointcalls* et non les flux/trajets des bateaux, les objets manipulés sont donc des indications d'escales entrantes ("In") ou sortantes ("Out"), avec des niveaux d'incertitude différents
* on compte parfois en pointcalls et parfois en nombre de `commodity_purposes` : ces métriques nous informent davantage sur la présence de données pour les objets étudiées que sur les quantités/valeurs effectives : il faut garder cette dimension en tête pour les interpréter correctement

In [ ]:
from poitousprint import Portic, get_pointcalls_port_as_toflit_partner, nest_portic_pointcall, get_pointcalls_commodity_purposes_as_toflit_product
import pandas as pds
from vega import VegaLite
from keplergl import KeplerGl

portic_client = Portic()
pointcalls = portic_client.get_pointcalls(source_subset='Poitou_1789', year=1789)

kepler_base_conf = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

## Aperçu des données sur les ports de la région

## Qu'est-ce qui rentre et qu'est-ce qui sort ?

In [ ]:
ports = {}
for p in pointcalls:
    port = p['pointcall']
    action = p['pointcall_action'].lower()
    in_region = p['pointcall_admiralty'] in ['Marennes', 'La Rochelle', "Sables-d’Olonne"]
    if port not in ports:
        lat = float(p['latitude']) if p['latitude'] is not None else None
        lon = float(p['longitude']) if p['longitude'] is not None else None
        ports[port] = {
            'in_region': 'amirautés de Marennes, La Rochelle et Sables d\'Olonne' if in_region else p['state_1789_fr'],
            'nb_pointcalls': 1,
            'latitude': lat,
            'longitude': lon,
            'ferme_bureau': p['ferme_bureau'],
            'etat': p['state_1789_fr']
        }
        ports[port][action] = 1
    else:
        ports[port]['nb_pointcalls'] += 1
        if action in ports[port]:
            ports[port][action] += 1
        else:
            ports[port][action] = 1
ports = [{'port': port, **vals} for port, vals in ports.items()]

points_map = KeplerGl(config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "6e7g5tt",
          "type": "point",
          "config": {
            "dataId": "data_1",
            "label": "Point",
            "color": [
              241,
              92,
              23
            ],
            "columns": {
              "lat": "latitude",
              "lng": "longitude",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Uber Viz Qualitative 1.2",
                "type": "qualitative",
                "category": "Uber",
                "colors": [
                  "#12939A",
                  "#DDB27C",
                  "#88572C",
                  "#FF991F",
                  "#F15C17",
                  "#223F9A"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                1,
                20
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 10,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "in_region",
              "type": "string"
            },
            "colorScale": "ordinal",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": {
              "name": "nb_pointcalls",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "data_1": [
              {
                "name": "port",
                "format": None
              },
              {
                "name": "in_region",
                "format": None
              },
              {
                "name": "nb_pointcalls",
                "format": None
              },
              {
                "name": "ferme_bureau",
                "format": None
              },
              {
                "name": "etat",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 45.39153346341695,
      "longitude": -1.155246746228874,
      "pitch": 0,
      "zoom": 4.451272741107292,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "light",
      "topLayerGroups": {
        "label": False,
        "building": False,
        "water": True,
        "land": False
      },
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}, data={'data_1': pds.DataFrame(ports)})
points_map

In [ ]:
# Lister les ports des 3 amirautés
local_pointcalls = [pointcall for pointcall in pointcalls if pointcall['pointcall_admiralty'] in ['Marennes', 'La Rochelle', "Sables-d’Olonne"]]

for p in local_pointcalls:
    p['pointcall_action'] = p['pointcall_action'].lower()

VegaLite({
    "title": "Distribution des pointcalls de la région étudiée",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_action"
        }
    }
}, pds.DataFrame(local_pointcalls))

In [ ]:
# Lister les ports hors des 3 amirautés
not_local_pointcalls = [pointcall for pointcall in pointcalls if pointcall['pointcall_admiralty'] not in ['Marennes', 'La Rochelle', "Sables-d’Olonne"]]

for p in not_local_pointcalls:
    p['pointcall_action'] = p['pointcall_action'].lower()

VegaLite({
    "title": "Distribution des pointcalls hors région, par ports",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_action"
        }
    }
}, pds.DataFrame(not_local_pointcalls))

## distribution des destinations

In [ ]:
VegaLite({
    "title": "Destinations des bateaux allant à l'extérieur de la région (pointcalls In hors région)",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity"
        }
    }
}, pds.DataFrame([p for p in not_local_pointcalls if p["pointcall_action"] == 'in']))

In [ ]:
VegaLite({
    "title": "Nombre de pointcalls pour des bateaux venant depuis l'extérieur de la région",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity",
            "title": "Incertitude"
        }
    }
}, pds.DataFrame([p for p in not_local_pointcalls if p["pointcall_action"] == 'out']))

In [ ]:
VegaLite({
    "title": "Destinations des bateaux allant à l'extérieur de la région (pointcalls 'In' hors région)",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall_as_toflit_partner",
            "type": "nominal",
            "title": "pointcalls par partenaire toflit grouping",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity",
            "title": "Incertitude"
        }
    }
}, pds.DataFrame([p for p in get_pointcalls_port_as_toflit_partner(not_local_pointcalls, partner_classification='partner_grouping') if p["pointcall_action"] == 'in']))

In [ ]:
VegaLite({
    "title": "Nombre de pointcalls pour des bateaux venant depuis l'extérieur de la région",
    "width": 800,
    "mark": "bar",
    "encoding": {
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de pointcalls"
        },
        "y": {
            "field": "pointcall_as_toflit_partner",
            "type": "nominal",
            "title": "pointcalls par partenaire toflit grouping",
            "sort": "-x"
        },
        "color": {
            "type": "nominal",
            "field": "pointcall_uncertainity",
            "title": "Incertitude"
        }
    }
}, pds.DataFrame([p for p in get_pointcalls_port_as_toflit_partner(not_local_pointcalls, partner_classification='partner_grouping') if p["pointcall_action"] == 'out']))

## Première vue sur les produits

In [ ]:
# todo affiner quand on aura commodity_action
local_in = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(local_pointcalls, "product_revolutionempire") if p["pointcall_action"] == 'in']
purposes_in = []
for pointcall in local_in:
    purposes_in += [{"port": pointcall["pointcall"], "uncertainty": pointcall["pointcall_uncertainity"], **p} for p in pointcall['commodity_purposes']]

VegaLite({
    "title": "Commodity_purposes (produits ou activités) arrivant dans la région (classification révolution & empire)",
    "width": 600,
    "mark": "bar",
    "encoding": {
        "y": {
            "type": "nominal",
            "field": "commodity_as_toflit",
            "title": "commodity purposes agrégés",
            "sort": "-x"
        },
        "x": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "color": {
            "type": "nominal",
            "field": "uncertainty"
        }
    }
}, pds.DataFrame(purposes_in))


In [ ]:
# todo affiner quand on aura commodity_action
local_in = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(local_pointcalls, "product_revolutionempire") if p["pointcall_action"] == 'in']
purposes_in = []
for pointcall in local_in:
    purposes_in += [{"port": pointcall["pointcall"], "admiralty": pointcall["pointcall_admiralty"], **p} for p in pointcall['commodity_purposes']]

VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) arrivant dans la région, par ports (classification révolution & empire)",
    "width": 600,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "produit",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_in))


In [ ]:
# todo affiner quand on aura commodity_action
local_out = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(local_pointcalls, "product_revolutionempire") if pointcall['pointcall_action'] == 'out']
purposes_out = []
for pointcall in local_out:
    purposes_out += [{"port": pointcall["pointcall"], "uncertainty": pointcall["pointcall_uncertainity"], **p} for p in pointcall['commodity_purposes']]


VegaLite({
"title": "Commodity_purposes (produits ou activités) sortant de la région (classification révolution & empire)",
"width": 600,
"mark": "bar",
"encoding": {
    "y": {
        "type": "nominal",
        "field": "commodity_as_toflit",
        "title": "commodity purposes agrégés",
        "sort": "-x"
    },
    "x": {
        "type": "quantitative", 
        "aggregate": "count",
        "title": "nombre de commodity purposes"
    },
    "color": {
        "type": "nominal",
        "field": "uncertainty"
    }
}
}, pds.DataFrame(purposes_out))


In [ ]:
VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) sortant de la région, par ports (classification révolution & empire)",
    "width": 600,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "commodity purposes (agrégés)",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_out))

In [ ]:
# todo affiner quand on aura commodity_action
global_in = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(not_local_pointcalls, "product_revolutionempire") if pointcall['pointcall_action'] == 'in']
purposes_in = []
for pointcall in global_in:
    purposes_in += [{"port": pointcall["pointcall"], **p} for p in pointcall['commodity_purposes']]


VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) envoyés depuis la région vers l'extérieur, par ports (classification révolution & empire)",
    "width": 1500,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "produit",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_in))

In [ ]:
# todo affiner quand on aura commodity_action
global_out = [pointcall for pointcall in get_pointcalls_commodity_purposes_as_toflit_product(not_local_pointcalls, "product_revolutionempire") if pointcall['pointcall_action'] == 'out']
purposes_out = []
for pointcall in global_out:
    purposes_out += [{"port": pointcall["pointcall"], "admiralty": pointcall["pointcall_admiralty"], **p} for p in pointcall['commodity_purposes']]


VegaLite({
    "title": "Nombre de commodity_purposes (produits ou activités) arrivant dans la région depuis l'extérieur, par ports (classification révolution & empire)",
    "width": 600,
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal",
            "field": "port",
            "sort": "-size"
        },
        "size": {
            "type": "quantitative", 
            "aggregate": "count",
            "title": "nombre de commodity purposes"
        },
        "y": {
            "field": "commodity_as_toflit",
            "type": "nominal",
            "title": "produit",
            "sort": "-size"
        }
    }
}, pds.DataFrame(purposes_out))
